<a href="https://colab.research.google.com/github/youssef893/Arabic-text-summarization/blob/main/arabic_text_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install transformers

In [2]:
!pip install transformers[torch]

In [3]:
# !pip install accelerate>=0.20.1

In [4]:
!pip install accelerate -U

In [5]:
!pip install datasets

In [6]:
!pip install SentencePiece

In [7]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import re
import nltk
from nltk.tokenize import word_tokenize

from datasets import Dataset as ds

from torch.utils.data import Dataset, DataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
import torch

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
data = pd.read_csv('/content/drive/MyDrive/wikiHow.csv')

In [10]:
data.head()

,summary,text
0,تناول الفاكهة والخضراوات في موسمها. تعرف على أ...,يكون سعر الفاكهة والخضراوات في موسم إنباتها أق...
1,فضل خيارات الأطعمة الأرخص ثمنا. تباطأ في استهل...,الأطعمة الصحية ليست باهظة الثمن بالضرورة، بل ف...
2,ازرع كل ما يمكنك من خضراوات وفاكهة. اطه بنفسك ...,استفد من حديقتك المنزلية أو أصيص الزرع الصغير ...
3,خطط مسبقا لوجباتك الرئيسية لمدة أسبوع. التزم ب...,تساعدك الخطط المسبقة في كل نواحي حياتك على وضع...
4,قلل وقت التعرض للشاشات. اذهب إلى الطبيب.,نظرا لأن السبب الرئيسي لضغط العين هو أن ثقافة ...


In [11]:
data['summary'][0]

'تناول الفاكهة والخضراوات في موسمها. تعرف على أسعار الأطعمة المجمدة والمعبئة. تابع العروض الأسبوعية في المحلات الكبرى. قارني دائما ما بين الأسعار. اشتر السلع بالجملة (كميات كبيرة). استفد من قسائم الشراء (كوبونات التخفيض). استفد من الجهات الداعمة لمحدودي الدخل في بلدك. اشتر أطعمتك من المزارع المحلية أو المتاجر التابعة لها.'

In [12]:
data['text'][0]

'يكون سعر الفاكهة والخضراوات في موسم إنباتها أقل من غيره من المواسم، وستلجأ محلات الخضروات إلى عرض الفاكهة بأسعار مناسبة في موسمها بسبب توفر المنتجات وزيادة الطلب عليها خلال تلك الفترات. لا يقتصر الأمر على السعر الأقل، بل سيكون طعامك من الخضراوات والفاكهة أشهى وألذ عند تناوله في موسمه.   في فصل الخريف: التفاح والتين والبنجر والكمثرى والقرنبيط والكرنب واليقطين في فصل الشتاء: (الخضراوات) الملفوف والفاصوليا والبازلاء والبصل (الفواكه)  البطاطا الحلوة والأفوكادو والبرتقال والتفاح والموز واليوسفى والرمان والعنب. في فصل الربيع: السبانخ والجزر والكوسة والبصل الأخضر  والطماطم والخضراوات الورقية والفراولة والمشمش. في فصل الصيف: الصيف هو فصل البطيخ، وكذلك الأمر بالنسبة للذرة والتوت. سوف تلاحظ توفر هذه الخضراوات والفواكهة المهمة على مدار العام كله بشكل أو آخر، ولكنها تكون بأرخص أسعارها في فصل الصيف، وهو ما يمكنك من شراء كميات كبيرة منها وتخزينها أو تجميدها في المجمد (الفريزر). الأفضل لصحتك البدنية والمالية على حد سواء هو تناول الفواكه والخضراوات والأطعمة الطازجة عامة، ولكن في بعض الحالات قد يكون م

In [13]:
# Remove duplicate rows
data = data.drop_duplicates()

In [14]:
data.head()

,summary,text
0,تناول الفاكهة والخضراوات في موسمها. تعرف على أ...,يكون سعر الفاكهة والخضراوات في موسم إنباتها أق...
1,فضل خيارات الأطعمة الأرخص ثمنا. تباطأ في استهل...,الأطعمة الصحية ليست باهظة الثمن بالضرورة، بل ف...
2,ازرع كل ما يمكنك من خضراوات وفاكهة. اطه بنفسك ...,استفد من حديقتك المنزلية أو أصيص الزرع الصغير ...
3,خطط مسبقا لوجباتك الرئيسية لمدة أسبوع. التزم ب...,تساعدك الخطط المسبقة في كل نواحي حياتك على وضع...
4,قلل وقت التعرض للشاشات. اذهب إلى الطبيب.,نظرا لأن السبب الرئيسي لضغط العين هو أن ثقافة ...


In [15]:
nltk.download('punkt')

def preprocess_arabic_text(text):
    # Remove non-Arabic characters and punctuation
    text = re.sub(r'[^\w\s\u0600-\u06FF]', '', text)
    text = re.sub(r'\s+', ' ', text)

    return text

data['text'] = data['text'].apply(preprocess_arabic_text)
data['summary'] = data['summary'].apply(preprocess_arabic_text)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [16]:
data.head()

,summary,text
0,تناول الفاكهة والخضراوات في موسمها تعرف على أس...,يكون سعر الفاكهة والخضراوات في موسم إنباتها أق...
1,فضل خيارات الأطعمة الأرخص ثمنا تباطأ في استهلا...,الأطعمة الصحية ليست باهظة الثمن بالضرورة، بل ف...
2,ازرع كل ما يمكنك من خضراوات وفاكهة اطه بنفسك و...,استفد من حديقتك المنزلية أو أصيص الزرع الصغير ...
3,خطط مسبقا لوجباتك الرئيسية لمدة أسبوع التزم بت...,تساعدك الخطط المسبقة في كل نواحي حياتك على وضع...
4,قلل وقت التعرض للشاشات اذهب إلى الطبيب,نظرا لأن السبب الرئيسي لضغط العين هو أن ثقافة ...


In [17]:
data['summary'][0]

'تناول الفاكهة والخضراوات في موسمها تعرف على أسعار الأطعمة المجمدة والمعبئة تابع العروض الأسبوعية في المحلات الكبرى قارني دائما ما بين الأسعار اشتر السلع بالجملة كميات كبيرة استفد من قسائم الشراء كوبونات التخفيض استفد من الجهات الداعمة لمحدودي الدخل في بلدك اشتر أطعمتك من المزارع المحلية أو المتاجر التابعة لها'

In [18]:
data['text'][0]

'يكون سعر الفاكهة والخضراوات في موسم إنباتها أقل من غيره من المواسم، وستلجأ محلات الخضروات إلى عرض الفاكهة بأسعار مناسبة في موسمها بسبب توفر المنتجات وزيادة الطلب عليها خلال تلك الفترات لا يقتصر الأمر على السعر الأقل، بل سيكون طعامك من الخضراوات والفاكهة أشهى وألذ عند تناوله في موسمه في فصل الخريف التفاح والتين والبنجر والكمثرى والقرنبيط والكرنب واليقطين في فصل الشتاء الخضراوات الملفوف والفاصوليا والبازلاء والبصل الفواكه البطاطا الحلوة والأفوكادو والبرتقال والتفاح والموز واليوسفى والرمان والعنب في فصل الربيع السبانخ والجزر والكوسة والبصل الأخضر والطماطم والخضراوات الورقية والفراولة والمشمش في فصل الصيف الصيف هو فصل البطيخ، وكذلك الأمر بالنسبة للذرة والتوت سوف تلاحظ توفر هذه الخضراوات والفواكهة المهمة على مدار العام كله بشكل أو آخر، ولكنها تكون بأرخص أسعارها في فصل الصيف، وهو ما يمكنك من شراء كميات كبيرة منها وتخزينها أو تجميدها في المجمد الفريزر الأفضل لصحتك البدنية والمالية على حد سواء هو تناول الفواكه والخضراوات والأطعمة الطازجة عامة، ولكن في بعض الحالات قد يكون من الأرخص بالنسبة لك 

In [19]:
len(data)

29200

In [20]:
train, ytrain = data['text'][:1000], data['summary'][:1000]
validation, y_validation = data['text'][20280:26800], data['summary'][2028:2680]
test, y_test = data['text'][2680:3000], data['summary'][2680:3000]

In [21]:
# Load the pre-trained T5 model and tokenizer
model_name = 't5-small'
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [22]:
class WikiDataset(Dataset):
    def __init__(self, df, target):
        self.texts = df
        self.labels = target

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = tokenizer(
            text, label,
            padding='max_length', max_length=128, truncation=True,
            return_tensors="pt"
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': encoding['input_ids'].squeeze()  # Assuming the label is the target text
        }

In [23]:
train = WikiDataset(train, ytrain)
val = WikiDataset(validation, y_validation)
test = WikiDataset(test, y_test)

In [24]:
training_args = TrainingArguments(
    output_dir='./output',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,  # Use Dataset object here
    eval_dataset =val
)

In [25]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, ove

Step,Training Loss


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

TrainOutput(global_step=375, training_loss=0.1365731404622396, metrics={'train_runtime': 89.7694, 'train_samples_per_second': 33.419, 'train_steps_per_second': 4.177, 'total_flos': 101506351104000.0, 'train_loss': 0.1365731404622396, 'epoch': 3.0})

In [27]:
trainer.evaluate(eval_dataset=test)

KeyError: ignored

In [ ]:
import nltk
from nltk.translate.bleu_score import corpus_bleu

def calculate_bleu_score(candidate_corpus, reference_corpus):
    """
    Calculate BLEU score for a candidate corpus against reference corpus.

    Args:
        candidate_corpus (list of lists): List of candidate sentences where each sentence is a list of tokens.
        reference_corpus (list of list of lists): List of reference sentences where each reference is a list of tokens.

    Returns:
        bleu_score (float): BLEU score.
    """
    assert len(candidate_corpus) == len(reference_corpus), "Number of sentences in candidate and reference should match"

    references = [[ref] for ref in reference_corpus]
    candidate_corpus = [cand.split() for cand in candidate_corpus]

    bleu_score = corpus_bleu(references, candidate_corpus)
    return bleu_score